In [1]:
import pandas as pd
import numpy as np
import datetime

import sys
sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")


In [ ]:
questions = {
    'Clustering':[
        'How can I mathematically Explain a Cluster.',
        'What is important.',
        'How can I determine what to drop, and when.',
        'How do I understand time components in Clusters.'
        ' The right way to think about “sweet spot”. The “sweet spot” is not: maximum variance explained, maximum number of features, maximum complexity It is: stable clusters, interpretable differences, consistent behavior across runs, sensitivity that matches your use case (short-term vs long-term behavior)',
        'Why ratios can be problematic in clustering.',
        'Ratios are: high signal, high risk. They:amplify noise, can dominate PCA, are hard to interpret. Rule: Try models with and without ratios Prefer simpler models unless ratios clearly add value. Ratios must earn their place.'
        
    ]

}




In [3]:


# --- Example usage ---
# 1) Keep only numeric columns
num_cols = cc_model_df.select_dtypes(include=[np.number]).columns

# 2) (Recommended) Standardize
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(cc_model_df[num_cols]), columns=num_cols, index=cc_model_df.index)

# 3) Compute VIF
vif_table = compute_vif(X_scaled)
print(vif_table)

### Reviewing Variables for Model Inclusion

def set_manual_column_order(df,column_name,column_order):
    
    '''
    
    '''
    
    df[column_name] = pd.Categorical(df[column_name], categories=column_order, ordered=True)

import timeit
from UtilityFunctions import PauseProcess
from eda_dq import column_statistical_review

def data_preparation_checklist(df=pd.DataFrame(),word_list=[]):
    if len(df)==0:
        df = pd.read_csv(links['d_learning_notes_url'])

    final_df = pd.DataFrame()
    
    for word in word_list:
        temp_df = df[(df['Definition'].fillna("").str.contains(word,case=False))|(df['Categorization']==word)]
        final_df = pd.concat([final_df,temp_df])

    final_df = final_df.drop(['Source','Process','Categorization'],axis=1)
    display(final_df)

    return final_df

def build_member_snapshot(final_mbr_df: pd.DataFrame,
                          final_txn_df: pd.DataFrame,
                          snapshot_month=None,
                          window_3m=3,
                          window_6m=6) -> pd.DataFrame:
    m = final_mbr_df.copy()
    t = final_txn_df.copy()

    # normalize month to month-start timestamp
    m["MONTH"] = _month_floor(m["MONTH"])
    t["MONTH"] = _month_floor(t["MONTH"])

    if snapshot_month is None:
        snapshot_month = m["MONTH"].max()
    snapshot_month = pd.to_datetime(snapshot_month).to_period("M").to_timestamp()

    # restrict to history up to snapshot month
    m = m[m["MONTH"] <= snapshot_month]
    t = t[t["MONTH"] <= snapshot_month]

    # ----- Member-month features windows -----
    # Keep a lean set of signals from your schema
    base_cols = [
        "MEMBERNBR","MONTH","AGE","CITY","BRANCHNAME","PRIMARY_IS_BEEM","HOME_OWNER",
        "ANNUAL_INCOME","DEPOSIT_BALANCE","MORTGAGE_BALANCE","LIQUID_ASSETS",
        "ENGAGEMENT_SCORE","PERC_DEPOSIT","PERC_MORTGAGE",
        "POS_TXN_DEBIT","BILL_PAY_DEBIT","PAYROLL_DEPOSIT","OTHER_DEPOSIT"
    ]
    base_cols = [c for c in base_cols if c in m.columns]
    m = m[base_cols].sort_values(["MEMBERNBR","MONTH"])

    # helper: window filter
    start_3m = snapshot_month - pd.DateOffset(months=window_3m-1)
    start_6m = snapshot_month - pd.DateOffset(months=window_6m-1)

    m3 = m[m["MONTH"].between(start_3m, snapshot_month)]
    m6 = m[m["MONTH"].between(start_6m, snapshot_month)]

    # 3m aggregates (means/sums)
    agg_3m = m3.groupby("MEMBERNBR").agg(
        age=("AGE","last"),
        city=("CITY","last"),
        branch=("BRANCHNAME","last"),
        primary=("PRIMARY_IS_BEEM","last"),
        home_owner=("HOME_OWNER","last"),
        income_mean_3m=("ANNUAL_INCOME","mean"),
        deposit_mean_3m=("DEPOSIT_BALANCE","mean"),
        loan_mean_3m=("MORTGAGE_BALANCE","mean"),
        engagement_mean_3m=("ENGAGEMENT_SCORE","mean"),
        pos_sum_3m=("POS_TXN_DEBIT","sum"),
        bill_sum_3m=("BILL_PAY_DEBIT","sum"),
        payroll_sum_3m=("PAYROLL_DEPOSIT","sum"),
        otherdep_sum_3m=("OTHER_DEPOSIT","sum"),
    ).reset_index()

    # 6m dynamics: volatility + trend (slope)
    def dyn_features(df, col):
        return df.groupby("MEMBERNBR")[col].agg(
            **{f"{col.lower()}_std_6m":"std",
               f"{col.lower()}_slope_6m":_slope}
        )

    dep_dyn = m6.groupby("MEMBERNBR")["DEPOSIT_BALANCE"].agg(
        deposit_std_6m="std",
        deposit_slope_6m=_slope
    )
    loan_dyn = m6.groupby("MEMBERNBR")["MORTGAGE_BALANCE"].agg(
        loan_std_6m="std",
        loan_slope_6m=_slope
    )
    eng_dyn = m6.groupby("MEMBERNBR")["ENGAGEMENT_SCORE"].agg(
        engagement_std_6m="std",
        engagement_slope_6m=_slope
    )

    dyn = pd.concat([dep_dyn, loan_dyn, eng_dyn], axis=1).reset_index()

    # ----- Transaction mix features (from your long txn table) -----
    # Build 3m spend mix + entropy (behavioral fingerprint)
    t3 = t[t["MONTH"].between(start_3m, snapshot_month)].copy()
    # aggregate category totals per member
    cat = t3.groupby(["MEMBERNBR","variable"])["value"].sum().reset_index()
    total = cat.groupby("MEMBERNBR")["value"].sum().rename("txn_total_3m").reset_index()
    cat = cat.merge(total, on="MEMBERNBR", how="left")
    cat["share"] = np.where(cat["txn_total_3m"] > 0, cat["value"] / cat["txn_total_3m"], 0.0)

    # pivot shares wide: share_<category>
    mix = cat.pivot_table(index="MEMBERNBR", columns="variable", values="share", fill_value=0.0)
    mix.columns = [f"share_{c}" for c in mix.columns]
    mix = mix.reset_index()

    # entropy: higher = more diverse mix
    # (avoid log(0) by masking)
    share_mat = mix.drop(columns=["MEMBERNBR"]).to_numpy(dtype=float)
    with np.errstate(divide="ignore", invalid="ignore"):
        ent = -(share_mat * np.log(np.where(share_mat > 0, share_mat, 1.0))).sum(axis=1)
    mix["mix_entropy_3m"] = ent

    # ----- Combine -----
    snap = agg_3m.merge(dyn, on="MEMBERNBR", how="left").merge(mix, on="MEMBERNBR", how="left")

    # convenience derived features
    snap["net_worth_mean_3m"] = snap["deposit_mean_3m"].fillna(0) - snap["loan_mean_3m"].fillna(0)
    snap["snapshot_month"] = snapshot_month

    # fill missing mix features with 0 (e.g., no txns in window)
    share_cols = [c for c in snap.columns if c.startswith("share_")]
    snap[share_cols] = snap[share_cols].fillna(0.0)
    for c in ["deposit_std_6m","deposit_slope_6m","loan_std_6m","loan_slope_6m",
              "engagement_std_6m","engagement_slope_6m","mix_entropy_3m"]:
        if c in snap.columns:
            snap[c] = snap[c].fillna(0.0)

    return snap


from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler, FunctionTransformer
from sklearn.pipeline import Pipeline

def make_preprocess_pipeline(feature_df: pd.DataFrame,
                             cat_cols=("city","branch"),
                             force_drop=("MEMBERNBR","snapshot_month")):
    df = feature_df.copy()

    # numeric candidates: everything number-ish except ids
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    num_cols = [c for c in num_cols if c not in force_drop]

    cat_cols = [c for c in cat_cols if c in df.columns]

    # log-transform for heavy-tailed money-ish cols
    log_cols = [c for c in num_cols if any(k in c for k in ["deposit","loan","income","pos_","bill_","payroll","otherdep","net_worth"])]
    passthrough_cols = [c for c in num_cols if c not in log_cols]

    log_pipe = Pipeline([
        ("log1p", FunctionTransformer(lambda x: np.log1p(np.clip(x, 0, None)))),
        ("scaler", RobustScaler())
    ])

    num_pipe = Pipeline([
        ("scaler", RobustScaler())
    ])

    pre = ColumnTransformer(
        transformers=[
            ("log_num", log_pipe, log_cols),
            ("num", num_pipe, passthrough_cols),
            ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ],
        remainder="drop"
    )
    return pre

# final_mbr_df, final_txn_df = generate_synthetic_dataset(...)

snap = build_member_snapshot(final_mbr_df, final_txn_df, snapshot_month=None, window_3m=3, window_6m=6)

feature_snap = snap.drop(columns=['MEMBERNBR','snapshot_month'],errors='ignore')
member_ids = snap['MEMBERNBR'].to_numpy()

pre = make_preprocess_pipeline(feature_snap)
X = pre.fit_transform(feature_snap)

print(X.shape)

# Cant run because it errors out, would be 22 Billion . Need to Understand this
#S = cosine_similarity(X)

from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=51,metric='cosine',algorithm='auto')

nn.fit(X)

member_index = {mid: i for i, mid in enumerate(member_ids)}

def test_knn(X,nn,k=10,sample_size=100,seed=42):
    rng = np.random.default_rng(seed)
    idxs = rng.choice(X.shape[0],size=sample_size,replace=False)

    # returns the distance and Index
    nn_dist,nn_index = nn.kneighbors(X[idxs],n_neighbors=k+1)
    nn_dist = nn_dist[:,1:].mean(axis=1)
    
    rand_idxs = rng.choice(X.shape[0],size=(sample_size,k),replace=True)
    rand_dist = np.linalg.norm(X[idxs][:,None,:]-X[rand_idxs],axis=2).mean(axis=1)
    
    return nn_dist,rand_dist,idxs


a,b,c = test_knn(X,nn)

mid = int(member_ids[0])
neighbors, sim = knn_topk(nn, member_index, member_ids, X, mid, k=10)
list(zip(neighbors.tolist(), sim.tolist()))


##########################################################################

# What does StandardScaler do. Transforms data to Mean 0 and Standard Dev = 1\

def Heatmap(df,
            correlation=True,
            column_list=[],
            title='Heat Map of Correlation',
            cmap='coolwarm',
            annotate=True,
            x_rotate=0,
            y_rotate=0,
            cbar=True,
            set_center=0,
            figsize=(10,10)):
    
    '''
    Function Which Generates a Heatmap
    
    Parameters:
        Dataframe
        column_name (list): If included, will only show certain columns on the Horizontal Axis.
    
    Returns:
        matlplot plot.
    
    '''
    
    sns.set(style='white')
    
    # View column with Abbreviated title or full. Abbreviated displays nicer.
    if correlation:
        corr = df.corr()
    else:
        corr = df.copy()
    
    if len(column_list)!=0:
        corr = corr[column_list]
    
    mask= np.zeros_like(corr,dtype=bool)
    mask[np.triu_indices_from(mask)]=True
    f,ax = plt.subplots(figsize=figsize)
    
    if len(str(set_center))!=0:
        sns.heatmap(corr,mask=mask,cmap=cmap,center=set_center,square=True,linewidths=1,annot=annotate,cbar=cbar)
    else:
        sns.heatmap(corr,mask=mask,cmap=cmap,square=True,linewidths=1,annot=annotate,cbar=cbar)
    
    
    plt.title(title)
    if y_rotate !=0:
        for tick in ax.get_yticklabels():
            tick.set_rotation(0)
            tick.set_horizontalalignment('right')
    if x_rotate !=0:
        plt.xticks(rotation=x_rotate,ha='center', va='top')

    plt.show()
